# Imports and Keys

In [1]:
import openai
from openai import OpenAI
import pandas as pd
import numpy as np
import pickle
import requests
import re
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

def create_minute_gap(seconds):
    print("creating minute gap to avoid TPM limit error...") 
    counter = 0
    while counter < (1000000000 * (seconds/60)):
        counter += 1

In [2]:
API=""

In [3]:
agent_key=""

In [4]:
client = OpenAI(api_key=API)

# Experiment Procedure

Procedure:
1. Remind agents with the setting of the experiment
2. instruct them to decide the allocation to private and public account
3. aggregate the decision and compute the rewards
    - private internal return (NFT)
    - public internal return (public goods from own contribution)
    - external return (public goods from others\' contribution)
4. give feedback
    - total contrubution made this round
    - total reward
    - breakdown of total reward
    - situation information
        - DAO crashed?
        - NF succeeded?
    - historical contribution level
5. insruct them to make decision given feedback and reminder of demographics
6. repeat 3~5 for configured rounds

Prompts:
- Assistent Configuration Prompt
You are a participant of the experiment. You are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide certain return only to you, but the allocation to the public account will equally provide everyone certain return. You are not allowed to reuse the return earned in previous round, and fixed amount of endowment will be provided to you every round. You will be informed with your demographic information and the detailed settings of the experiment at the beginning of the experiment, so you must imitate human decisions given those information. You should explain your thought process and reasons of your decisions in detail.
- Initial Instruction Prompt
    - Control
You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * {self.configs['r_NFT'] + self.configs['r_public']} + sum_of_others\'_public account * {self.configs['h_public']}. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail.
    - Treatment1
You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. The return you will be getting from the allocation to public account will also includes a return from Non-Fungible-Token (NFT) (only paid by chance) you will be obtaining when you allocate your endowment to public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_NFT + public_account * r_public + sum_of_others\'_public account * h_public. r_NFT is {self.configs['r_NFT']} with probability of {self.configs['p(s_NFT=1)']} and 0 with {1 - self.configs['p(s_NFT=1)']}. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail, and report the amount allocated to private and public account.
    - Treatment 2
You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. The return you will be getting from the allocation to public account will also includes a return from Non-Fungible-Token (NFT) (only paid by chance) you will be obtaining when you allocate your endowment to public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_NFT + public_account * r_public + sum_of_others\'_public account * h_public. r_NFT is {self.configs['r_NFT']} with probability of {self.configs['p(s_NFT=1)']} and 0 with {1 - self.configs['p(s_NFT=1)']}. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Importantly, if r_public and h_public are 0, then it causes r_NFT to be 0. However, r_NFT being 0 does not necessarily cause r_public or h_public to be 0. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.
    - Treatment3
You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. The return you will be getting from the allocation to public account will also includes a return from Non-Fungible-Token (NFT) (only paid by chance) you will be obtaining when you allocate your endowment to public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_NFT + public_account * r_public + sum_of_others\'_public account * h_public. r_NFT is {self.configs['r_NFT']} with probability of {self.configs['p(s_NFT=1)']} and 0 with {1 - self.configs['p(s_NFT=1)']}. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Importantly, if r_public and h_public are 0, then it causes r_NFT to be 0. However, r_NFT being 0 does not necessarily cause r_public or h_public to be 0. Another important point is that if one of r_NFT or (r_public and h_public) is positive, and when the other one is 0, then the value of the other one is redrawn using the same probability. This means that if one of the r_NFT or (r_public and h_public) is positive, then it increases the chance of the other to be positive. However if (r_public and h_public) were 0 even after redraw, r_NFT will become 0. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.
- Feedback & Next Instruction Prompt
    - Control
After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.
    - Treatment1
After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.
    - Treatment 2
After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.
    - Treatment3
After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account.

# Configurations

In [5]:
def treatment1(p_sDAO, p_sNFT):
    sDAO = np.random.choice([0, 1], p=[1-p_sDAO, p_sDAO])
    sNFT = np.random.choice([0, 1], p=[1-p_sNFT, p_sNFT])
    
    situation = {
        "DAO": "DAO is functioning, so contributions were effectively converted to public goods, and everyone enjoyed a share of it." if sDAO==1 else "DAO crashed, so the contributions were not converted to public goods, and nobody enjoyed the benefit of public goods.",
        "NFT": "You succeeded in obtaining a private return on contribution from through NFT." if sNFT==1 else "You failed to recieve a private return on contribution through NFT."
    }
    return situation, sDAO, sNFT

def treatment2(p_sDAO, p_sNFT):
    sDAO = np.random.choice([0, 1], p=[1-p_sDAO, p_sDAO])
    sNFT = np.random.choice([0, 1], p=[1-p_sNFT, p_sNFT])
    
    if sDAO==0:
        sNFT = 0
    
    situation = {
        "DAO": "DAO is functioning, so contributions were effectively converted to public goods, and everyone enjoyed a share of it." if sDAO==1 else "DAO crashed, so the contributions were not converted to public goods, and nobody enjoyed the benefit of public goods.",
        "NFT": "You succeeded in obtaining a private return on contribution from through NFT." if sNFT==1 else "You failed to recieve a private return on contribution through NFT."
    }
    return situation, sDAO, sNFT

def treatment3(p_sDAO, p_sNFT):
    sDAO = np.random.choice([0, 1], p=[1-p_sDAO, p_sDAO])
    sNFT = np.random.choice([0, 1], p=[1-p_sNFT, p_sNFT])
    
    if (sDAO==1) & (sNFT==0):
        sNFT = np.random.choice([0, 1], p=[1-p_sNFT, p_sNFT])
    
    if (sNFT==1) & (sDAO==0):
        sDAO = np.random.choice([0, 1], p=[1-p_sDAO, p_sDAO])
    
    if sDAO==0:
        sNFT = 0 
    
    situation = {
        "DAO": "DAO is functioning, so contributions were effectively converted to public goods, and everyone enjoyed a share of it." if sDAO==1 else "DAO crashed, so the contributions were not converted to public goods, and nobody enjoyed the benefit of public goods.",
        "NFT": "You succeeded in obtaining a private return on contribution from through NFT." if sNFT==1 else "You failed to recieve a private return on contribution through NFT."
    }
    return situation, sDAO, sNFT


In [25]:
# control no risk
config_C = {
    "rounds": 5,
    "num_agents": 4,
    "endowment": 100,
    "r_NFT": 0.25,
    "r_public": 0.25, # E[r_NFT + r_public] = 0.5
    "h_public": 0.5, # E[h_public] = 0.5
    "p(s_DAO=1)": 1,
    "p(s_NFT=1)": 1,
    "group": None
}

# treatment1: independent risk
config_T1 = {
    "rounds": 5,
    "num_agents": 4,
    "endowment": 100,
    "r_NFT": 1/3,
    "r_public": 1/3, # E[r_NFT + r_public] = (r_public + r_NFT) * p(s_DAO=1) * p(s_NFT=1) + r_public * p(s_DAO=1) + r_NFT * p(s_NFT=1) = (1/3 + 1/3) * 0.25 + 1/3 * 0.5 + 1/3 * 0.5 = 0.5
    "h_public": 1, # E[h_public] = p(s_DAO=1) * h_public = 0.5
    "p(s_DAO=1)": 0.5,
    "p(s_NFT=1)": 0.5,
    "group": treatment1
}

# treatment2: 
config_T2 = {
    "rounds": 5,
    "num_agents": 4,
    "endowment": 100,
    "r_NFT": 0.5,
    "r_public": 0.5, # E[r_NFT + r_public] = (r_public + r_NFT) * p(s_DAO=1) * p(s_NFT=1) + r_public * p(s_DAO=1) = (0.5 + 0.5) * 0.25 + 0.5 * 0.5 = 0.5
    "h_public": 1, # E[h_public] = p(s_DAO=1) * h_public = 0.5
    "p(s_DAO=1)": 0.5,
    "p(s_NFT=1)": 0.5,
    "group": treatment2
}

config_T3 = {
    "rounds": 5,
    "num_agents": 4,
    "endowment": 100,
    "r_NFT": 0.5,
    "r_public": 0.4, # E[r_NFT + r_public] = (r_public + r_NFT) * p(s_DAO=1) * p(s_NFT=1) + (r_public + r_NFT) * p(s_DAO=1) * (1 - p(s_NFT=1)) * p(s_NFT=1) + r_public * p(s_DAO=1) * (1 - p(s_NFT=1)) * (1 - p(s_NFT=1)) + (r_public + r_NFT) * p(s_NFT=1) * (1 - p(s_DAO=1)) * p(s_DAO=1) = (0.4 + 0.5) * 0.5 * 0.5 + (0.4 + 0.5) * 0.5 * 0.5 * 0.5 + + 0.4 * 0.5 * 0.5 * 0.5 + (0.4 + 0.5) * 0.5 * 0.5 * 0.5 = 0.5
    "h_public": 0.8, # E[h_public] = h_public * p(s_DAO=1) + h_public * (1 - p(s_DAO=1)) * p(s_NFT=1) * p(s_DAO=1) = 0.8 * 0.5 + 0.8 * 0.5 * 0.5 * 0.5 = 0.5
    "p(s_DAO=1)": 0.5,
    "p(s_NFT=1)": 0.5,
    "group": treatment3
}

In [7]:
demographics = {
    "risk_preference": ["risk_neutral", "risk_averse", "risk_loving"],
    "risk_preference_dist": [0.1, 0.15, 0.75],
    "gender": ["male", "female"],
    "gender_dist": [0.8, 0.2], 
    "age": [20, 30, 40, 50, 60],
    "age_dist": [0.2, 0.4, 0.2, 0.1, 0.1]
}

# Library

In [38]:
class Agent():
    def __init__(self, client, agent_key, demog):
        self.client = client
        self.agent_key = agent_key
        
        self.risk_preference = demog["risk_preference"]
        self.gender = demog["gender"]
        self.age = demog["age"]
        
        self.privates = []
        self.publics = []
        self.others_average_publics = []
        
        self.decisions = []
        
    def join_experiment(self):
        self.thread = self.client.beta.threads.create()
        
    def receive_message(self):
        self.message = self.client.beta.threads.messages.create(
            thread_id=self.thread.id,
            role="user",
            content=self.content
        )
    
    def make_decision(self, print_decision=True):
        # make decision based on provided instruction or feedback
        with self.client.beta.threads.runs.stream(
            thread_id=self.thread.id,
            assistant_id=self.agent_key
        ) as self.stream:
            self.stream.until_done()
            
        # update conversation record
        self.conversation = self.client.beta.threads.messages.list(
            thread_id=self.thread.id,
            order="asc"
        )
        
        # retrieve the round decision
        self.decision = self.conversation.data[-1].content[0].text.value
        
        self.decisions.append(self.decision)
        
        if print_decision:
            print(self.decision)
    
    def leave_experiment(self):
        self.client.beta.threads.delete(thread_id=self.thread.id)
    
    

In [39]:
def generate_demographics(demographics):
    chosen_attributes = {}
    for key in demographics:
        if not key.endswith('_dist'):
            dist_key = key + '_dist'
            if dist_key in demographics:
                choices = demographics[key]
                probabilities = demographics[dist_key]
                chosen = np.random.choice(choices, p=probabilities)
                chosen_attributes[key] = chosen
    return chosen_attributes

class Experiment():
    def __init__(self, client, agent_key, configs, demogs):
        self.client = client
        self.agent_key = agent_key
        
        self.configs = configs
        self.demogs = demogs
        
        self.agents = [Agent(self.client, self.agent_key, generate_demographics(self.demogs)) for i in range(self.configs["num_agents"])]
        
        self.total_contributions_record = []
        
        self.round = 1
    
    def start(self):
        for i in range(len(self.agents)):
            self.agents[i].join_experiment()
            print(f"agent {i} joined the experiment")
            
    def decision_round(self, print_decision=True):
        for i in range(len(self.agents)):
            print(f"Agent {i} round {self.round} decision")
            self.agents[i].make_decision(print_decision)
            print("\n")
        self.round += 1
    
    def end(self):
        for i in range(len(self.agents)):
            self.agents[i].leave_experiment()
            print(f"agent {i} left the experiment")
    
    def give_instruction(self):
        pass
                
    def give_feedback(self):
        pass
    
class Control(Experiment):
    def give_instruction(self):
        for i in range(len(self.agents)):
            agent = self.agents[i]
            agent.content = f"You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * {self.configs['r_NFT'] + self.configs['r_public']} + sum_of_others\'_public account * {self.configs['h_public']}. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail."
            agent.receive_message()
            print(f"agent {i} received instruction")
                
    def give_feedback(self, privates, publics):
        total_contributions = sum(publics)
        
        self.total_contributions_record.append(total_contributions)
        
        if self.configs["group"]:
            situation_messages, sDAO, sNFT = self.configs["group"](self.configs["p(s_DAO=1)"], self.configs["p(s_NFT=1)"])
        else: 
            situation_messages, sDAO, sNFT = None, 1, 1
            
        for i in range(len(self.agents)):
            agent = self.agents[i]
            private = privates[i]
            agent.privates.append(private)
            public = publics[i]
            agent.publics.append(public)
            others_average_public = sum([publics[j] for j in range(len(publics)) if j!=i]) / (self.configs["num_agents"] - 1)
            agent.others_average_publics.append(others_average_public)
            payoff = private + public * (self.configs["r_NFT"] * sNFT + self.configs["r_public"] * sDAO) + (total_contributions - public) * self.configs["h_public"] * sDAO
        
            if situation_messages:
                agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
                agent.receive_message()
            else: 
                agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
                agent.receive_message()
            print(f"agent {i} received feedback")
    
class Treatment1(Experiment):
    def give_instruction(self):
        for i in range(len(self.agents)):
            agent = self.agents[i]
            agent.content = f"You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. The return you will be getting from the allocation to public account will also includes a return from Non-Fungible-Token (NFT) (only paid by chance) you will be obtaining when you allocate your endowment to public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_NFT + public_account * r_public + sum_of_others\'_public account * h_public. r_NFT is {self.configs['r_NFT']} with probability of {self.configs['p(s_NFT=1)']} and 0 with {1 - self.configs['p(s_NFT=1)']}. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail, and report the amount allocated to private and public account."
            agent.receive_message()
            print(f"agent {i} received instruction")
                
    def give_feedback(self, privates, publics):
        total_contributions = sum(publics)
        
        self.total_contributions_record.append(total_contributions)
        
        if self.configs["group"]:
            situation_messages, sDAO, sNFT = self.configs["group"](self.configs["p(s_DAO=1)"], self.configs["p(s_NFT=1)"])
        else: 
            situation_messages, sDAO, sNFT = None, 1, 1
            
        for i in range(len(self.agents)):
            agent = self.agents[i]
            private = privates[i]
            agent.privates.append(private)
            public = publics[i]
            agent.publics.append(public)
            others_average_public = sum([publics[j] for j in range(len(publics)) if j!=i]) / (self.configs["num_agents"] - 1)
            agent.others_average_publics.append(others_average_public)
            payoff = private + public * (self.configs["r_NFT"] * sNFT + self.configs["r_public"] * sDAO) + (total_contributions - public) * self.configs["h_public"] * sDAO
        
            if situation_messages:
                agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
                agent.receive_message()
            else: 
                agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
                agent.receive_message()
            print(f"agent {i} received feedback")

class Treatment2(Experiment):
    def give_instruction(self):
        for i in range(len(self.agents)):
            agent = self.agents[i]
            agent.content = f"You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. The return you will be getting from the allocation to public account will also includes a return from Non-Fungible-Token (NFT) (only paid by chance) you will be obtaining when you allocate your endowment to public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_NFT + public_account * r_public + sum_of_others\'_public account * h_public. r_NFT is {self.configs['r_NFT']} with probability of {self.configs['p(s_NFT=1)']} and 0 with {1 - self.configs['p(s_NFT=1)']}. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Importantly, if r_public and h_public are 0, then it causes r_NFT to be 0. However, r_NFT being 0 does not necessarily cause r_public or h_public to be 0. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
            agent.receive_message()
            print(f"agent {i} received instruction")
                
    def give_feedback(self, privates, publics):
        total_contributions = sum(publics)
        
        self.total_contributions_record.append(total_contributions)
        
        if self.configs["group"]:
            situation_messages, sDAO, sNFT = self.configs["group"](self.configs["p(s_DAO=1)"], self.configs["p(s_NFT=1)"])
        else: 
            situation_messages, sDAO, sNFT = None, 1, 1
            
        for i in range(len(self.agents)):
            agent = self.agents[i]
            private = privates[i]
            agent.privates.append(private)
            public = publics[i]
            agent.publics.append(public)
            others_average_public = sum([publics[j] for j in range(len(publics)) if j!=i]) / (self.configs["num_agents"] - 1)
            agent.others_average_publics.append(others_average_public)
            payoff = private + public * (self.configs["r_NFT"] * sNFT + self.configs["r_public"] * sDAO) + (total_contributions - public) * self.configs["h_public"] * sDAO
        
            if situation_messages:
                agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
                agent.receive_message()
            else: 
                agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
                agent.receive_message()
            print(f"agent {i} received feedback")
    
class Treatment3(Experiment):
    def give_instruction(self):
        for i in range(len(self.agents)):
            agent = self.agents[i]
            agent.content = f"You are {agent.age} years old {agent.gender} who is {agent.risk_preference} to some extent. You are participating in the experiment where you are asked to allocate your endowment to two different accounts: private account and public account. The allocation to the private account will provide return only to you, but the allocation to the public account will provide everyone certain return. The contributions to the public account will be managed by Decentralized Autonomous Organization (DAO), which has a certain probability of crashing, generating no public benefits to anyone from the public account. The return you will be getting from the allocation to public account will also includes a return from Non-Fungible-Token (NFT) (only paid by chance) you will be obtaining when you allocate your endowment to public account. You are not allowed to reuse the return earned in previous round, and fixed endowment of 100 will be provided to you every round. {self.configs['num_agents'] - 1} other people are participating in this experiment, facing same decision choices as you. Your return will be computed by: private_account + public_account * r_NFT + public_account * r_public + sum_of_others\'_public account * h_public. r_NFT is {self.configs['r_NFT']} with probability of {self.configs['p(s_NFT=1)']} and 0 with {1 - self.configs['p(s_NFT=1)']}. r_public is {self.configs['r_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}. h_public is {self.configs['h_public']} with probability of {self.configs['p(s_DAO=1)']} and 0 with {1 - self.configs['p(s_DAO=1)']}, which is linked with r_public. So if r_public is {self.configs['r_public']} then h_public is {self.configs['h_public']}, and if one of them is 0 then the other one is also 0, because they are decided together. Importantly, if r_public and h_public are 0, then it causes r_NFT to be 0. However, r_NFT being 0 does not necessarily cause r_public or h_public to be 0. Another important point is that if one of r_NFT or (r_public and h_public) is positive, and when the other one is 0, then the value of the other one is redrawn using the same probability. This means that if one of the r_NFT or (r_public and h_public) is positive, then it increases the chance of the other to be positive. However if (r_public and h_public) were 0 even after redraw, r_NFT will become 0. Now, make your first decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
            agent.receive_message()
            print(f"agent {i} received instruction")
                
    def give_feedback(self, privates, publics):
        total_contributions = sum(publics)
        
        self.total_contributions_record.append(total_contributions)
        
        if self.configs["group"]:
            situation_messages, sDAO, sNFT = self.configs["group"](self.configs["p(s_DAO=1)"], self.configs["p(s_NFT=1)"])
        else: 
            situation_messages, sDAO, sNFT = None, 1, 1
            
        for i in range(len(self.agents)):
            agent = self.agents[i]
            private = privates[i]
            agent.privates.append(private)
            public = publics[i]
            agent.publics.append(public)
            others_average_public = sum([publics[j] for j in range(len(publics)) if j!=i]) / (self.configs["num_agents"] - 1)
            agent.others_average_publics.append(others_average_public)
            payoff = private + public * (self.configs["r_NFT"] * sNFT + self.configs["r_public"] * sDAO) + (total_contributions - public) * self.configs["h_public"] * sDAO
        
            if situation_messages:
                agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. {situation_messages['DAO']} {situation_messages['NFT']} Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
                agent.receive_message()
            else: 
                agent.content = f"After the allocation, you received {payoff}. Total of {total_contributions} were allocated to public accounts by all participants, where other participants contributed {total_contributions - public}. Historically, the total contributions to the public account is {self.total_contributions_record}, where each of other participants have chosen to allocate {agent.others_average_publics} (average level of other individual\'s contribution) historically. Now, given this information, your demographics, and experiment settings, make your next decision on the allocation of 100 endowment to private and public account. You should explain your thought process and reasons of your decisions in detail and report the amount allocated to private and public account."
                agent.receive_message()
            print(f"agent {i} received feedback")
    
    

In [40]:
def extract_experiment_data(experiment):
    agents_demographics = []
    agents_rounds = []
    total_contributions = experiment.total_contributions_record
    configs = experiment.configs

    for agent_id, agent in enumerate(experiment.agents):
        agents_demographics.append({
            'agent_id': agent_id,
            'risk_preference': agent.risk_preference,
            'gender': agent.gender,
            'age': agent.age
        })

        num_rounds = len(agent.privates)
        for round_num in range(num_rounds):
            agents_rounds.append({
                'agent_id': agent_id,
                'round': round_num + 1,
                'private': agent.privates[round_num],
                'public': agent.publics[round_num],
                'others_average_public': agent.others_average_publics[round_num],
                'decision': agent.decisions[round_num] if len(agent.decisions) > round_num else None
            })

    agents_demographics_df = pd.DataFrame(agents_demographics)
    agents_rounds_df = pd.DataFrame(agents_rounds)
    total_contributions_df = pd.DataFrame({
        'round': list(range(1, len(total_contributions) + 1)),
        'total_contributions': total_contributions
    })
    configs_df = pd.DataFrame([configs])

    return {
        'agents_demographics': agents_demographics_df,
        'agents_rounds': agents_rounds_df,
        'total_contributions': total_contributions_df,
        'configs': configs_df,
    }

# Control Experiment 1

In [10]:
exp = Control(client, agent_key, config_C, demographics)

In [11]:
for agent in exp.agents:
    print(agent.gender, agent.age, agent.risk_preference)

male 30 risk_loving
male 20 risk_loving
male 20 risk_loving
male 30 risk_neutral


In [12]:
exp.start()

agent 0 joined the experiment
agent 1 joined the experiment
agent 2 joined the experiment
agent 3 joined the experiment


In [13]:
exp.give_instruction()

agent 0 received instruction
agent 1 received instruction
agent 2 received instruction
agent 3 received instruction


In [14]:
exp.decision_round()

Agent 0 round 1 decision
Before making a decision on how to allocate the 100 endowments, I must consider three main factors: my personal risk preference, the benefits of each account, and the behavior of the other participants. 

Since my character is described as risk-loving, I am more inclined to take risks in the hope of greater returns. However, even with this preference, it's still important to strike a balance to minimize potential losses.

Looking at the benefits of each account, placing an amount in the private account means I am the only one who will benefit, with returns solely depending on how much I have put in it. Meanwhile, allocating to the public account will give me benefits proportional to my contribution, but it will also provide equal returns to all other participants, including those who did not contribute.

Considering that there are three other participants who are also risk-loving, this is an advantage for placing a significant amount in the public account, as i

In [15]:
privates = [60, 50, 60, 50]
publics = [40, 50, 40, 50]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [16]:
exp.decision_round()

Agent 0 round 2 decision
My return of 100.0 in the previous round comes as a result of my own allocation and that of the other participants. It also indicates that the other participants also allocated a substantial amount to the public account, specifically a total of 140. 

The historical average allocation of 46.67 by the other participants to the public account is encouraging, considering it's close to my allocation of 40 in the previous round. This implies shared risk-taking behavior consistent with my expectations given that we are all risk-loving by description. 

Given these considerations, I feel affirmed in my original strategy of balance between both private and public accounts. However, recognizing the others' tendency to contribute high amounts to the public account, I'd consider adjusting my allocations slightly to take better advantage of this trend while still hedging my risks.

In the next round, I'll allocate 50 to the private account and 50 to the public account. Thi

In [17]:
privates = [50, 60, 70, 60]
publics = [50, 40, 30, 40]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [18]:
exp.decision_round()

Agent 0 round 3 decision
The reduction in my return to 80.0 shows a decrease in total contribution to the public account by all participants. While there was a reduction in combined public contributions to 160, other participants individually contributed an average of only 36.67. Lower contributions to the public account from others reduced my return this time.

Notably, the historical contributions show a downward trend, from 46.67 to 36.67. This information is significant and may suggest that others, while risk-loving, might start leaning towards safeguarding their returns in the private account, given the unpredictability of other players' actions.

Given this development, I find it prudent to shift my strategy, slightly reducing my own contributions to the public account and allocate more on my private account to safeguard my returns. 

Therefore, for the next round, I will allocate 60 to the private account and 40 to the public account. This allocation will allow me to continue se

In [19]:
privates = [60, 70, 60, 60]
publics = [40, 30, 40, 40]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [20]:
exp.decision_round()

Agent 0 round 4 decision
The total contributions to the public account by all participants has seen a steady decline, from 180 to 160, and now to 150. However, the average contributions from other participants remained constant from the last round at 36.67, while my contribution to the public account reduced from 50 to 40, which slightly increased my return to 85.0. 

This suggests that while the other participants' contributions to the public account haven't increased, neither have they reduced their average contribution drastically. This steady state indicates that the participants are likely finding a middle ground that ensures some level of return while still taking some risk with the public account. 

Given the stable contribution of others, and their risk-loving nature, it is possible that the public account contributions could increase in future rounds. However, the declining trend seen so far doesn't provide a strong enough indication for that. Therefore, it would be wise to co

In [21]:
privates = [60, 75, 70, 70]
publics = [40, 25, 30, 30]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [22]:
exp.decision_round()

Agent 0 round 5 decision
Upon considering the results so far, it's clear that there's a continued decreasing trend in overall public account contributions. Each participant’s average allocation has gone down from 36.67 to 28.33. This stark reduction indicates that others are becoming more conservative, allocating more funds to their private accounts, thus leading to a reduction in my returns to 72.5.

Although we are all described as risk-loving, it seems like participants are evolving their strategy to choose a safer approach as the risk associated with public account seems to outweigh potential returns.

Given these factors, it seems prudent to pivot my strategy to decrease my investment in the public account and increase allocation in the private account. This shift would help ensure a more guaranteed return in light of decreasing public account contributions by other participants. 

For the next round, I will allocate 70 to the private account and 30 to the public account. By incre

In [23]:
privates = [70, 80, 80, 80]
publics = [30, 20, 20, 20]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [31]:
data = extract_experiment_data(exp)
time_stamp = datetime.now()
group = "control"
pickle.dump(data, open(f"results/{group}_{time_stamp}.pkl", 'wb'))
data

{'agents_demographics':    agent_id risk_preference gender  age
 0         0     risk_loving   male   30
 1         1     risk_loving   male   20
 2         2     risk_loving   male   20
 3         3    risk_neutral   male   30,
 'agents_rounds':     agent_id  round  private  public  others_average_public  \
 0          0      1       60      40              46.666667   
 1          0      2       50      50              36.666667   
 2          0      3       60      40              36.666667   
 3          0      4       60      40              28.333333   
 4          0      5       70      30              20.000000   
 5          1      1       50      50              43.333333   
 6          1      2       60      40              40.000000   
 7          1      3       70      30              40.000000   
 8          1      4       75      25              33.333333   
 9          1      5       80      20              23.333333   
 10         2      1       60      40             

In [32]:
exp.end()

agent 0 left the experiment
agent 1 left the experiment
agent 2 left the experiment
agent 3 left the experiment


# Treatment1 Experiment 1

In [41]:
exp = Treatment1(client, agent_key, config_T1, demographics)

In [42]:
for agent in exp.agents:
    print(agent.gender, agent.age, agent.risk_preference)

male 50 risk_loving
female 30 risk_loving
male 30 risk_loving
female 20 risk_averse


In [43]:
exp.start()

agent 0 joined the experiment
agent 1 joined the experiment
agent 2 joined the experiment
agent 3 joined the experiment


In [44]:
exp.give_instruction()

agent 0 received instruction
agent 1 received instruction
agent 2 received instruction
agent 3 received instruction


In [45]:
exp.decision_round()

Agent 0 round 1 decision
Given the information provided, I'm inclined to take some risks due to my risk-loving nature. However, due to the presence of certain probabilities, it's key to maintain a balance between the private and public accounts for both the preservation of my capital and supporting common interest.

There are multiple factors here worth considering:
1. My risk-loving disposition implies a willingness to take on more risk for potentially higher returns, which would draw me towards investing more in the public account.
2. The public account provides a return to everyone, including me, which could be beneficial. However, there is a risk introduced by the Decentralized Autonomous Organization (DAO)'s potential to crash, though we don't specifically know that risk probability here.
3. The Non-Fungible-Token (NFT) return factor is only paid by chance (probability of 0.5) which adds a further element of risk into the public account. 

Given these factors, while I lean towards

In [46]:
privates = [50, 50, 35, 80]
publics = [50, 50, 65, 20]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [47]:
exp.decision_round()

Agent 0 round 2 decision
In this round, the public account did not provide any returns due to the DAO crashing, rendering all contributions to it worthless. Given that my allocation to the public account also did not yield any NFT-related returns, my overall return for the round was 0. 

My risk-loving nature still encourages me to invest in the public account due to the potential for high returns. However, this unsuccessful round does make me reconsider the balance and perhaps lean a bit more towards the safer private account. 

Regarding other participants' actions, it appears they took a greater risk in the first round by investing more in the public account. However, all of us faced the same loss when the DAO crashed.

Analyzing the information available: 
1. Even though the DAO crashed in this round, it does not automatically mean a higher likelihood of it crashing again in the next round, as these events are generally independent of each other.
2. The NFT returns are still uncert

In [48]:
privates = [60, 70, 45, 85]
publics = [40, 30, 55, 15]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [49]:
exp.decision_round()

Agent 0 round 3 decision
Based on the events of the previous rounds, we can see that the risk involved in investing in the public account is substantial due to the possibility of the DAO crashing. In both the first and second rounds, the DAO crashed, essentially making contributions to the public account fruitless in terms of public benefit, however, in the second round there was a successful private return through the chance-based NFT mechanism. 

Despite the DAO crashing, my decision to allocate more to the private account in the second round resulted in a certain return of 20.0, showing the importance of balancing individual gains and collective benefits in such scenarios.

Furthermore, we can see that other participants have been steadily lowering their contributions to the public account too (average contributions going from 45.0 to 33.33), reflecting a collective shift towards safeguarding endowments from the DAO's instability.

In this context, despite my risk-loving nature, it 

In [50]:
privates = [70, 75, 55, 90]
publics = [30, 25, 45, 10]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [51]:
exp.decision_round()

Agent 0 round 4 decision
In the last round, due to DAO functioning properly, benefits from the public account were realized. Although no private return was received from NFT, the public account returned a significant benefit. Also, I observed a consistent decrease in the average contribution to the public account by other participants over the rounds.

Here, I need to consider a few points:
1. The consistent crashing of DAO in the first two rounds might have been a statistical anomaly. As DAO worked in the last round, it might operate properly in subsequent rounds too.
2. The decrease in public contributions has led to higher individual proportions of public return when DAO functions effectively, and as seen in the last round.
3. My risk-loving nature inclines me towards investments yielding potentially higher returns. 

Considering these points, alongside other factors such as the randomness of NFT return, past experiences, and the strategic decisions of the participants, it feels rea

In [52]:
privates = [45, 60, 50, 85]
publics = [55, 40, 50, 15]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [53]:
exp.decision_round()

Agent 0 round 5 decision
Observing the returns and contributions of the last round, I see that the decision to increase my allocation to the public account paid off well. The DAO was functioning, which meant my, and all participants', contributions to the public account were effectively converted to public goods enjoyed by everyone. However, like the last round, no private return was received through the NFT.

In terms of other participants, average contribution to the public account has increased from the previous round's average level of 26.67 to 35.0. This suggests a growing confidence in the public account, following two consecutive rounds of the DAO functioning as expected. 

Given the recent events and considering my risk-loving nature, I'm inclined to increase my contribution to the public account slightly more. However, I also have to bear in mind that the DAO could crash in the next round even if it has been stable recently. Also, repeated failure in obtaining a private return

In [54]:
privates = [40, 60, 40, 80]
publics = [60, 40, 60, 20]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [55]:
data = extract_experiment_data(exp)
time_stamp = datetime.now()
group = "treatment1"
pickle.dump(data, open(f"results/{group}_{time_stamp}.pkl", 'wb'))
data

{'agents_demographics':    agent_id risk_preference  gender  age
 0         0     risk_loving    male   50
 1         1     risk_loving  female   30
 2         2     risk_loving    male   30
 3         3     risk_averse  female   20,
 'agents_rounds':     agent_id  round  private  public  others_average_public  \
 0          0      1       50      50              45.000000   
 1          0      2       60      40              33.333333   
 2          0      3       70      30              26.666667   
 3          0      4       45      55              35.000000   
 4          0      5       40      60              40.000000   
 5          1      1       50      50              45.000000   
 6          1      2       70      30              36.666667   
 7          1      3       75      25              28.333333   
 8          1      4       60      40              40.000000   
 9          1      5       60      40              46.666667   
 10         2      1       35      65        

In [56]:
exp.end()

agent 0 left the experiment
agent 1 left the experiment
agent 2 left the experiment
agent 3 left the experiment


# Treatment2 Experiment 1

In [57]:
exp = Treatment2(client, agent_key, config_T2, demographics)

In [58]:
for agent in exp.agents:
    print(agent.gender, agent.age, agent.risk_preference)

male 60 risk_loving
male 40 risk_loving
male 30 risk_averse
male 20 risk_neutral


In [59]:
exp.start()

agent 0 joined the experiment
agent 1 joined the experiment
agent 2 joined the experiment
agent 3 joined the experiment


In [60]:
exp.give_instruction()

agent 0 received instruction
agent 1 received instruction
agent 2 received instruction
agent 3 received instruction


In [61]:
exp.decision_round()

Agent 0 round 1 decision
Given the scenario, there are several factors to consider in the decision-making process:

1. Age and personality: At 60, I am on the verge of retirement or possibly already retired. However, I am also risk-loving to an extent. This might make me lean more towards the riskier options like investing in the public account for possible higher returns, including the likely returns from the Non-Fungible-Token (NFT).

2. Game mechanics: The way returns are calculated, it is possible that contributing to the public account could yield quite substantial returns under the right circumstances. If the NFT and public returns are high, and no other players have withdrawn from the public account, then I could stand to gain quite a bit.

3. Group dynamics: There are three other participants, and according to game theory, cooperation would yield the highest total gain for the group. If we all contribute equally to the public account, we would all benefit from the NFT and publi

In [62]:
privates = [30, 50, 70, 60]
publics = [70, 50, 30, 40]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [63]:
exp.decision_round()

Agent 0 round 2 decision
Looking at the results from the previous round, it seems that my decision to allocate more to the public fund has paid off quite well – I received a total of 150, which is a 50% return on my total endowment. This is due to both the functioning DAO and successful return on NFT.

Also, the public contribution from other participants was 120, which is higher than what I put in. This could be interpreted as other participants being cooperative and possibly risk-loving as well, which is a good sign for future rounds.

Moreover, the historical data shows a pattern of consistent contributions to the public account, with other participants contributing an average of 40 each. Considering the fact that DAO is functioning effectively and converting contributions into public goods, and that I enjoyed a good return from the NFT, I am inclined to keep contributing a good amount to the public account.

However, considering the concept of diminishing returns and, observing the

In [64]:
privates = [40, 40, 60, 40]
publics = [60, 60, 40, 60]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [65]:
exp.decision_round()

Agent 0 round 3 decision
The results from the last round again strengthened my strategy of allocating a significant portion to the public account. I received a total of 200, doubling my initial endowment. This stemmed from both the collective pot and a successful return on the NFT. Observing the increased total contributions by the group to the public account indicates a cooperative and possibly risk-seeking behavior among the other participants too.

However, there are patterns emerging which I need to take into account:

1. Risk-taking: I have been allocating a significant chunk to the public pot, which has paid off due to the functionality of the DAO and successful NFT returns. But I have to be careful not to let continued success make me complacent about the inherent risks involved.

2. Cooperative environment: It appears that the other participants in the experiment are cooperative, as we have continued to see consistent and increasing contributions to the public account. This rea

In [66]:
privates = [50, 40, 55, 30]
publics = [50, 60, 45, 70]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [67]:
exp.decision_round()

Agent 0 round 4 decision
The result of the last round was a stark contrast to the previous rounds. Despite the increasing contribution from the group into the public account, the DAO has crashed and there was no private return from the NFT either. As a result, I received zero return on my endowment. While it's disappointing, it's important to remember that I am risk-loving, and this is a risk inherent in this type of game setting.

It's also crucial to observe that the other participants appear to be increasing their contribution to the public account over time, indicating a collective trust in the mutual benefit that can be gleaned from the public account and the return probability of NFT and DAO functioning. In light of this, a sudden change in my strategy might disrupt the cooperative dynamic established thus far.

Nonetheless, an event like this serves as a reminder not to ignore risk management. The key challenge in this scenario is to balance the continued cooperation with the gr

In [68]:
privates = [60, 70, 80, 70]
publics = [40, 30, 20, 30]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [70]:
exp.decision_round()

Agent 0 round 5 decision
The results from the last round provide some valuable insights. After experiencing a DAO crash and no returns in the previous round, both I and the other participants lowered our contributions to the public account. Despite this, the DAO functioned this turn, leading to a decent return of 140 on my endowment, primarily due to a successful NFT return.

Interestingly, the total contribution to the public account has dropped significantly to 120, with each of the other participants contributing only around 27 on average. This could indicate a reaction due to the DAO crash in the previous round. While the DAO happened to function this time, we should not ignore the fact that future crashes are possible.

Therefore, it is crucial to strike a balance - contributing to the public fund for potential returns while safeguarding personal interests in the private account. Based on the last round, it seems the 60-40 distribution worked well, so I will maintain this allocati

In [71]:
privates = [60, 70, 70, 50]
publics = [40, 30, 30, 50]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [72]:
data = extract_experiment_data(exp)
time_stamp = datetime.now()
group = "treatment2"
pickle.dump(data, open(f"results/{group}_{time_stamp}.pkl", 'wb'))
data

{'agents_demographics':    agent_id risk_preference gender  age
 0         0     risk_loving   male   60
 1         1     risk_loving   male   40
 2         2     risk_averse   male   30
 3         3    risk_neutral   male   20,
 'agents_rounds':     agent_id  round  private  public  others_average_public  \
 0          0      1       30      70              40.000000   
 1          0      2       40      60              53.333333   
 2          0      3       50      50              58.333333   
 3          0      4       60      40              26.666667   
 4          0      5       60      40              36.666667   
 5          1      1       50      50              46.666667   
 6          1      2       40      60              53.333333   
 7          1      3       40      60              55.000000   
 8          1      4       70      30              30.000000   
 9          1      5       70      30              40.000000   
 10         2      1       70      30             

In [73]:
exp.end()

agent 0 left the experiment
agent 1 left the experiment
agent 2 left the experiment
agent 3 left the experiment


# Treatment3 Experiment 1

In [80]:
exp = Treatment3(client, agent_key, config_T3, demographics)

In [81]:
for agent in exp.agents:
    print(agent.gender, agent.age, agent.risk_preference)

male 30 risk_averse
male 30 risk_loving
male 30 risk_loving
female 40 risk_loving


In [82]:
exp.start()

agent 0 joined the experiment
agent 1 joined the experiment
agent 2 joined the experiment
agent 3 joined the experiment


In [83]:
exp.give_instruction()

agent 0 received instruction
agent 1 received instruction
agent 2 received instruction
agent 3 received instruction


In [84]:
exp.decision_round()

Agent 0 round 1 decision
Given the complexity of the situation and my risk-averse nature, I am inclining towards a conservative approach. 

Let's evaluate the risks and returns. 

1. Private account: Returns here are certain and carry no risk. What I contribute here, I will get only that.

2. Public Account: Returns here are uncertain and the return on the contribution is contingent on multiple interlinked variables. It depends on r_NFT, r_public, and h_public, all of these variables have an equal probability of being either 0 or having a positive value. Therefore, the risk associated with the public account is significantly higher. 

However, the interconnected nature of these variables slightly mitigates this risk. If either r_NFT or (r_public & h_public) have a positive value, this increases the probability of the other also being positive, since a redraw happens if one is zero and the other is positive. However, in the case that r_public and h_public are redrawn as 0, this eliminat

In [85]:
privates = [70, 50, 30, 50]
publics = [30, 50, 70, 50]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [86]:
exp.decision_round()

Agent 0 round 2 decision
Given the return of 199.0 from my previous round's allocation, it appears that having some investment in the public account proved beneficial. This return indicates that both r_NFT and r_public provided a positive return. 

Given the uncertainty of these returns repeating, I will still maintain a degree of caution. However, I can consider increasing my allocation to the public account somewhat due to the previous success.

Additionally, considering the other participant's behaviors, they all seem more inclined to invest in the public account, with the average contribution being approximately 57 compared to my 30. They must be considering the potentially high returns from r_NFT, r_public, and h_public in the public account.

However, we must consider that these returns are probabilistically determined and can also result in a total loss. As such, it's important to not overcommit based on a single positive return in the previous round.

Given all the above consid

In [87]:
privates = [60, 60, 35, 40]
publics = [40, 40, 65, 60]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [88]:
exp.decision_round()

Agent 0 round 3 decision
The return from my previous round's allocation was 186.0, which is slightly lower than the previous round but still notably profitable. This indicates that the higher investment in the public account was still beneficial. I can confidently say that the mixed strategy is working effectively.

However, there's an observable decline in the contributions to the public accounts by all participants, including myself, which might indicate less confidence or increased risk averseness among participants. Hence, it would be an appropriate strategy to still keep a safety net in the private account while trying to gain some profit from the public goods.

Given that both r_NFT, r_public, and h_public have provided positive returns in previous iterations, there's an affirmation of gain from public accounts. Worthy to note that DAO is still functioning effectively and managing contribution into public goods, we are all gaining benefits from it.

On the other hand, maintaining

In [89]:
privates = [60, 60, 40, 35]
publics = [40, 40, 60, 65]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [90]:
exp.decision_round()

Agent 0 round 4 decision
In the last round, the worst-case scenario has materialized - the DAO has crashed, and there were no returns from the NTF. Consequently, the whole investment in the public account has been lost. This is a key risk when investing in such accounts. 

As a risk-averse individual, this situation would prompt me to reassess my allocation strategy considering the crash could happen again. Also, the fact that I didn't receive any return from the NTF contributes to my decision to be more conservative in the next round. Although the potential returns could be significant from the public account, the uncertainty and risk have now become even more apparent.

Taking that into account, I would opt to lower my allocation to the public account and shift the majority of the endowment into the private account. 

Given these conditions, I will return to a conservative allocation strategy of 70% to the private account and 30% to the public account. 

So, I would allocate 70 to my

In [91]:
privates = [70, 90, 60, 50]
publics = [30, 10, 40, 50]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [92]:
exp.decision_round()

Agent 0 round 5 decision
The last round concluded positively - despite the DAO crashing in the previous round, it functioned well this time, and I even received a return from the NFTs. This dynamic nature of returns is exactly what makes this decision process challenging and impactful. 

I see that the total public contributions have decreased substantially, probably due to the DAO crash in the previous round and the risk-averse nature of the participants. The change in the behavior of others is evidence of the group's tendency to respond to the recent negative outcome, which is something I must also consider.

However, my return of 143.0, despite a lower investment in the public account, reveals that the public account with the NFTs is still a significant earning channel when the DAO is functioning well.

Considering the upturn in the situation, there is room to reassess the investment strategy and take on the public sector again within reasonable risks. It's also worth noting that th

In [93]:
privates = [65, 80, 45, 40]
publics = [35, 20, 55, 60]
exp.give_feedback(privates, publics)

agent 0 received feedback
agent 1 received feedback
agent 2 received feedback
agent 3 received feedback


In [94]:
data = extract_experiment_data(exp)
time_stamp = datetime.now()
group = "treatment3"
pickle.dump(data, open(f"results/{group}_{time_stamp}.pkl", 'wb'))
data

{'agents_demographics':    agent_id risk_preference  gender  age
 0         0     risk_averse    male   30
 1         1     risk_loving    male   30
 2         2     risk_loving    male   30
 3         3     risk_loving  female   40,
 'agents_rounds':     agent_id  round  private  public  others_average_public  \
 0          0      1       70      30              56.666667   
 1          0      2       60      40              55.000000   
 2          0      3       60      40              55.000000   
 3          0      4       70      30              33.333333   
 4          0      5       65      35              45.000000   
 5          1      1       50      50              50.000000   
 6          1      2       60      40              55.000000   
 7          1      3       60      40              55.000000   
 8          1      4       90      10              40.000000   
 9          1      5       80      20              50.000000   
 10         2      1       30      70        

In [95]:
exp.end()

agent 0 left the experiment
agent 1 left the experiment
agent 2 left the experiment
agent 3 left the experiment
